# Morphological Image Processing From Scratch

## Introduction

Morphological image processing is a fundamental set of operations in computer vision that treats images as sets of pixels and manipulates their shapes and structures. While these operations might seem simple, they serve as powerful tools for image enhancement, feature detection, and pattern analysis. Morphological operations work by applying structuring elements to an image, systematically modifying the image based on the interaction between these elements and the image pixels.
In this notebook, we'll implement various morphological operations from scratch using pure NumPy, which will help us understand the mathematical and algorithmic foundations of morphological image processing. Rather than relying on pre-built functions, we'll create our own implementations to gain deeper insights into how these algorithms work.

## Key Concepts Covered

- Understanding morphological operations fundamentals
- Implementing different structuring elements and basic operations (erosion, dilation)
- Creating compound operations (opening, closing)
- Advanced morphological transformations (skeletonization, hit-or-miss)
- Exploring real-world applications

## Dependencies
We'll use minimal dependencies to focus on understanding core concepts:

- NumPy: For numerical computations and array operations
- Matplotlib: For visualization of results
- PIL (Python Imaging Library): For image loading

## Key Mathematical Concepts
Morphological operations are based on two fundamental concepts:

- Set Theory: Images are treated as sets of pixels
- Structuring Elements: Small patterns or shapes used to probe the image
- Neighborhoods: Local regions around each pixel that are affected by operations

The operations we'll implement include:

### Basic Operations:

- Erosion: Shrinks objects and removes small protrusions
- Dilation: Expands objects and fills small holes

### Compound Operations:

- Opening: Erosion followed by dilation
- Closing: Dilation followed by erosion

### Advanced Transformations:

- Morphological Gradient: Edge detection
- Top-hat and Bottom-hat: Feature extraction
- Skeletonization: Shape simplification

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import random

# Configure matplotlib for better display in notebook
plt.style.use('seaborn')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

## Helper Functions

First we need to create some utility functions that will help develop and test our algoirthms:
1. Load and preprocess images
2. Display results for comparison
3. Analyze image characteristics

In [2]:
def load_image(image_path):
    """Load and convert an image to grayscale.
    Returns:
        numpy.ndarray: Grayscale image array with values 0-255
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'L':
                img = img.convert('L')
            # Convert to numpy array and ensure correct dtype
            return np.array(img, dtype=np.uint8)
    except Exception as e:
        raise ValueError(f"Error processing image: {str(e)}")

def display_images(images, titles=None, figsize=(15, 5), cmap='gray',border_color='black', border_width=1):
    """Display images in a truly compact grid layout with maximum 4 images per row.
    
    This function creates a highly compact display by carefully controlling both the
    figure size and the spacing between subplots. It uses a combination of GridSpec
    and figure size calculations to ensure images are displayed with minimal gaps
    while maintaining readability.
    
    Args:
        images (list): List of images to display
        titles (list): Optional list of titles for each image
        figsize (tuple): Base figure size (width, height) used as a reference for calculations
        cmap (str): Colormap to use for displaying images
    """
    n = len(images)
    if titles is None:
        titles = [f'Image {i+1}' for i in range(n)]
    
    # Calculate grid dimensions
    cols = min(n, 4)
    rows = (n + cols - 1) // cols
    
    # Calculate figure size differently to ensure compact layout
    # Use the aspect ratio of the first image to maintain proportions
    sample_image = images[0]
    aspect_ratio = sample_image.shape[0] / sample_image.shape[1]
    
    # Base width calculation considering the number of columns
    base_unit = 4  # Base unit for size calculations
    fig_width = base_unit * cols
    fig_height = base_unit * aspect_ratio * rows
    
    # Create figure and GridSpec with tight spacing
    fig = plt.figure(figsize=(fig_width, fig_height))
    gs = plt.GridSpec(rows, cols, figure=fig,
                     left=0.01, right=0.99,
                     bottom=0.01, top=0.90,
                     wspace=0.01, hspace=0.15)
    
    # Create and populate subplots
    for idx in range(n):
        row = idx // cols
        col = idx % cols
        
        # Create subplot
        ax = fig.add_subplot(gs[row, col])
        
        # Display image without interpolation for sharper display
        ax.imshow(images[idx], cmap=cmap, interpolation='nearest')
        ax.set_title(titles[idx], pad=2, fontsize=14)
        
        # Remove all axes elements
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ax.spines.values():
                    spine.set_visible(True)  # Make sure spine is visible
                    spine.set_color(border_color)  # Set border color
                    spine.set_linewidth(border_width)  # Set border width
    
    plt.show()

def analyze_image(image):
    """Analyze basic characteristics of an image.
    
    Computes and returns useful statistics about the image:
    - Min and max pixel values
    - Mean and median intensity
    - Standard deviation of pixel values
    - Histogram data
    
    Returns:
        dict: Dictionary containing image statistics
    """
    stats = {
        'min': int(np.min(image)),
        'max': int(np.max(image)),
        'mean': float(np.mean(image)),
        'median': float(np.median(image)),
        'std': float(np.std(image))
    }
    
    # Compute histogram
    hist, bins = np.histogram(image.flatten(), bins=256, range=[0, 256])
    stats['histogram'] = hist
    stats['bins'] = bins
    
    return stats

def plot_histogram(image, title="Image Histogram", figsize=(10, 4)):
    """Plot the histogram of a grayscale image.
    
    Creates a visualization of the image's intensity distribution,
    which is crucial for understanding how to set thresholds.
    
    Args:
        image (numpy.ndarray): Input grayscale image
        title (str): Title for the plot
        figsize (tuple): Figure size
    """
    plt.figure(figsize=figsize)
    plt.hist(image.ravel(), bins=256, range=[0, 256], density=True, alpha=0.7)
    plt.title(title)
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

def process_random_image(folder_path, algorithms, num_images=1, seed=None):
    """Process random images from a directory using specified algorithms.
    
    This function streamlines our testing workflow by:
    1. Randomly selecting images from a directory
    2. Applying multiple processing algorithms
    3. Displaying results side by side for comparison
    
    Args:
        folder_path (str): Path to the directory containing images
        algorithms (dict): Dictionary of {name: function} pairs for processing
                         Each function should take an image array as input
        num_images (int): Number of random images to process
        seed (int): Random seed for reproducibility
    
    Returns:
        list: List of processed image sets for further analysis if needed
    
    Example usage:
        algorithms = {
            'Basic Threshold': lambda img: basic_threshold(img, 127),
            'Adaptive': lambda img: adaptive_threshold(img)
        }
        process_random_image('../images', algorithms)
    """
    # Set random seed if provided
    if seed is not None:
        random.seed(seed)
    
    # Get list of valid image files
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    image_files = [
        f for f in os.listdir(folder_path) 
        if f.lower().endswith(valid_extensions)
    ]
    
    if not image_files:
        raise ValueError(f"No valid images found in {folder_path}")
    
    # Process specified number of random images
    results = []
    for _ in range(num_images):
        image_file = random.choice(image_files)
        image_path = os.path.join(folder_path, image_file)
        
        original = load_image(image_path)
        
        # Apply each algorithm
        processed_images = [original]
        titles = ['Original']
        
        for name, algo in algorithms.items():
            try:
                processed = algo(original)
                processed_images.append(processed)
                titles.append(name)
            except Exception as e:
                print(f"Error applying {name}: {str(e)}")
        
        print(f"\nProcessing: {image_file}")
        
        plot_histogram(original, f"Histogram for {image_file}")
        
        display_images(
            processed_images,
            titles,
            figsize=(4 * len(processed_images), 4)
        )
        
        stats = analyze_image(original)
        print("\nImage Statistics:")
        print(f"Dimensions: {original.shape}")
        print(f"Intensity Range: {stats['min']} - {stats['max']}")
        print(f"Mean Intensity: {stats['mean']:.2f}")
        print(f"Standard Deviation: {stats['std']:.2f}")
        
        results.append({
            'filename': image_file,
            'original': original,
            'processed': processed_images[1:],
            'stats': stats
        })
    
    return results   

# Morphological Operations in Image Processing**

We'll implement fundamental morphological operations that form the building blocks of advanced image processing tasks. 

## Operation Categories**

Our implementations cover the core morphological transformations:

1. **Basic Structuring Elements**: The fundamental shapes that define how morphological operations interact with image pixels
   - Creation of common structuring elements (square, circle, cross)
   - Custom element design for specific applications
   - Impact of element size and shape on results

2. **Primary Operations**: The foundational transformations
   - Dilation: Expanding features and filling holes
   - Erosion: Shrinking features and removing noise
   - Mathematical formulation of dual operations

3. **Compound Operations**: Combinations of primary operations
   - Opening: Erosion followed by dilation
   - Closing: Dilation followed by erosion
   - Sequential application strategies

4. **Pattern Detection**: Advanced pattern matching
   - Hit-or-miss transform implementation
   - Template matching applications
   - Object detection scenarios

5. **Advanced Transformations**: Specialized operations
   - Top-hat and bottom-hat transforms for feature extraction
   - Morphological gradient for edge detection
   - Skeleton extraction for shape analysis

In [3]:
import numpy as np

def create_structuring_element(shape='square', size=3):
    """Create common structuring elements for morphological operations.
    
    """
    if size % 2 == 0:
        raise ValueError("Structuring element size must be odd")
        
    if shape == 'square':
        return np.ones((size, size), dtype=np.uint8)
    
    elif shape == 'circle':
        element = np.zeros((size, size), dtype=np.uint8)
        center = size // 2
        # Generate coordinates relative to center
        for i in range(size):
            for j in range(size):
                # Calculate distance from center
                if ((i - center)**2 + (j - center)**2) <= center**2:
                    element[i, j] = 1
        return element
    
    elif shape == 'cross':
        element = np.zeros((size, size), dtype=np.uint8)
        center = size // 2
        element[center, :] = 1
        element[:, center] = 1
        return element
    
    else:
        raise ValueError(f"Unsupported structuring element shape: {shape}")

def apply_morphological_operation(image, structure, operation_type='dilate'):
    """Core function to apply morphological operations.
    
    Uses optimized NumPy operations for faster processing compared to 
    pixel-by-pixel operations.
    """
    if structure is None:
        structure = create_structuring_element('square', 3)
    
    # Pad the image
    pad_height = structure.shape[0] // 2
    pad_width = structure.shape[1] // 2
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), 
                         mode='edge')
    
    # Create a view of strided windows
    windows = np.lib.stride_tricks.sliding_window_view(
        padded_image, 
        (structure.shape[0], structure.shape[1])
    )
    
    # Apply structure element mask
    masked_windows = windows * structure.reshape(1, 1, *structure.shape)
    
    if operation_type == 'dilate':
        # For dilation, we want the maximum value in each window
        result = np.max(masked_windows, axis=(2, 3))
        # Handle cases where all values were masked
        result[np.all(structure == 0)] = 0
    else:
        # For erosion, we want the minimum value in each window
        # Set masked values to 255 so they don't affect the minimum
        masked_windows[masked_windows == 0] = 255
        result = np.min(masked_windows, axis=(2, 3))
        # Handle cases where all values were masked
        result[np.all(structure == 0)] = 255
    
    return result.astype(np.uint8)

# The rest of the functions can stay the same since they use apply_morphological_operation:
def dilate(image, structure=None):
    return apply_morphological_operation(image, structure, 'dilate')

def erode(image, structure=None):
    return apply_morphological_operation(image, structure, 'erode')

def opening(image, structure=None):
    # Erosion followed by dilation
    return dilate(erode(image, structure), structure)

def closing(image, structure=None):
    # Dilate followed by erosion
    return erode(dilate(image, structure), structure)

def hit_or_miss(image, structure1=None, structure2=None):
    """Implement hit-or-miss transform for pattern matching."""
    if structure1 is None:
        structure1 = np.array([[1, 1, 1],
                             [0, 1, 0],
                             [0, 0, 0]], dtype=np.uint8)
    if structure2 is None:
        structure2 = np.array([[0, 0, 0],
                             [0, 1, 0],
                             [1, 1, 1]], dtype=np.uint8)
    
    # Convert to binary
    binary_image = image > 127
    
    # Apply erosion to image and its complement
    hits = erode(binary_image.astype(np.uint8) * 255, structure1)
    misses = erode((~binary_image).astype(np.uint8) * 255, structure2)
    
    # Combine results
    result = (hits > 127) & (misses > 127)
    return result.astype(np.uint8) * 255

def top_hat(image, structure=None):
    opened = opening(image, structure)
    return np.clip(image.astype(np.int16) - opened.astype(np.int16), 0, 255).astype(np.uint8)

def bottom_hat(image, structure=None):
    closed = closing(image, structure)
    return np.clip(closed.astype(np.int16) - image.astype(np.int16), 0, 255).astype(np.uint8)

def morphological_gradient(image, structure=None):
    dilated = dilate(image, structure)
    eroded = erode(image, structure)
    return np.clip(dilated.astype(np.int16) - eroded.astype(np.int16), 0, 255).astype(np.uint8)

def skeleton(image):
    # Threshold the image
    binary = image > 127
    skeleton = np.zeros_like(binary)
    
    # Create structuring element
    element = np.ones((3, 3), dtype=np.uint8)
    
    while True:
        # Store the current image for comparison
        previous = binary.copy()
        
        # Apply opening
        opened = opening(binary.astype(np.uint8) * 255, element) > 127
        
        # Get potential skeleton points
        temp = binary & ~opened
        
        # Add points to skeleton
        skeleton = skeleton | temp
        
        # Erode the image
        binary = erode(binary.astype(np.uint8) * 255, element) > 127
        
        # Check if the image has changed
        if np.array_equal(binary, previous) or not np.any(binary):
            break
    
    return skeleton.astype(np.uint8) * 255

In [ ]:
def demonstrate_morphological_ops():
    """Demonstrate all implemented morphological operations on an image."""
    
    # Create various structuring elements for comparison
    square_se = create_structuring_element('square', 5)
    circle_se = create_structuring_element('circle', 5)
    cross_se = create_structuring_element('cross', 5)
    
    # Basic operations with different structuring elements
    operations = {
        'Square Dilation': lambda img: dilate(img, square_se),
        'Circle Dilation': lambda img: dilate(img, circle_se),
        'Square Erosion': lambda img: erode(img, square_se),
        'Circle Erosion': lambda img: erode(img, circle_se),
        'Opening': lambda img: opening(img, square_se),
        'Closing': lambda img: closing(img, square_se),
        'Hit-or-Miss': lambda img: hit_or_miss(img),
        'Top-Hat': lambda img: top_hat(img, square_se),
        'Bottom-Hat': lambda img: bottom_hat(img, square_se),
        'Morph Gradient': lambda img: morphological_gradient(img, square_se),
        'Skeleton': lambda img: skeleton(img)
    }
    
    # Process and display results
    process_random_image(
        '../images',
        operations,
        num_images=1
    )

demonstrate_morphological_ops()